**Plot topography, pits and observed brightness temperature for each AOI**

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
import math
import rasterio
from rasterio.plot import show
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from rasterstats import gen_point_query
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.ops import unary_union

In [ ]:
%load_ext autoreload
%aimport aesop
%autoreload 1

Load data:

In [ ]:
topography_rast = rasterio.open('../Data/topography/TPIsimple.tif')

In [ ]:
pits = aesop.get_pit_topography()

In [ ]:
aoi_polygons = aesop.get_aoi_polygons()
aoi_polygons = aoi_polygons.to_crs(crs=topography_rast.crs.data)

In [ ]:
obs = aesop.get_obs_per_aoi('C087')

Get height observations for each AOI to use in calucating the instrument foorprints:

In [ ]:
data = aesop.slf_data('../Data/raw_airborne/metoffice-marss_faam_20180316_r001_c087.nc',0)

height_obs = gpd.GeoDataFrame(data.height, geometry=gpd.points_from_xy(data.longitude, data.latitude), columns=['height'],
                crs={'init':'epsg:4326'}).dropna().reset_index(drop=True).to_crs(crs=topography_rast.crs.data)

height_obs = pd.concat([height_obs.loc[height_obs.geometry.within(polygon)].reset_index(drop=True)
                        for polygon in aoi_polygons.geometry], keys=aoi_polygons.index)

for aoi, row in height_obs.groupby(level=0):
    topo = [t for t in gen_point_query(row.geometry, '../Data/topography/TPIsimple.tif', interpolate='nearest')]

    height_obs.loc[aoi, 'topography_id'] = topo
    height_obs['topography_name'] = height_obs['topography_id'].map({1.0:'plateau', 2.0:'valley', 3.0:'slopes'})

In [ ]:
def footprint(beamwidth, height):
    footprint = 2*(height*math.tan(math.radians(beamwidth)/2))
    return footprint

In [ ]:
def buffer(aoi):
    ac_height = height_obs.loc[aoi]
    FWHM_89 = 11.8
    footprint_89 = footprint(FWHM_89, ac_height.height)
    
    obs_89 = obs['M16-89'].loc[aoi]
    obs_89['footprint'] = footprint_89
    
    # For each point in the observations geometry apply a buffer where the radius is the footprint of the instrument at that point divided by 2.
    obs_89['geometry'] = obs_89.apply(
        lambda row: row.geometry.buffer(row.footprint/2), axis=1)

    buffer = unary_union(obs_89['geometry'])
    
    return buffer

Normalise colours across all observations:

In [ ]:
vmin = min([np.nanmin(n) for n in  obs['M16-89'].TB.values])
vmax = max([np.nanmax(n) for n in  obs['M16-89'].TB.values])
norm = colors.Normalize(vmin=vmin, vmax=vmax)

In [ ]:
# Aspect ratios taken for the largest pit
asp_x, asp_y = 2274.7741302622017, 2032.0736330915242
asp = asp_x/asp_y

In [ ]:
fig, axes = plt.subplots(3,3,figsize=(12,12))#, constrained_layout=True)

axes[2,2].remove()

channel = 'M16-89'
cmap = ListedColormap(['#CD853F', '#FFFF66', '#90EE90', '#3399FF'])

for (index, aoi), ax in zip(aoi_polygons.sort_index().iterrows(), axes.flatten()):   
    obs_aoi = obs[channel].loc[index]
    obs_plot = ax.scatter(obs_aoi.geometry.x, obs_aoi.geometry.y, c=obs_aoi.TB, 
                          cmap='plasma', norm=norm)
    
    ax.plot(*aoi.geometry.exterior.xy, color='#0066CC')
    aoi_temp = Rectangle([0,0], 1,1, edgecolor='#0066CC', facecolor='none', linewidth=2, 
                         label='Areas of Interest')

    ymin, ymax = ax.get_ylim()
    xmin, xmax = ax.get_xlim()
    
    rasterio.plot.show(topography_rast, ax=ax, cmap=cmap, alpha=0.8, 
                       label='Topographic index')
    pits[pits.index.str.contains(index)].plot(marker='P', c='#33FFFF', edgecolor='k', 
                                              ax=ax, markersize=140, label='Pits', 
                                              legend=True)

    asp_x_diff = (asp_x-(xmax-xmin))/2
    asp_y_diff = (asp_y-(ymax-ymin))/2
    
    ax.set_xlim(xmin-asp_x_diff, xmax+asp_x_diff)
    ax.set_ylim(ymin-asp_y_diff, ymax+asp_y_diff)
    
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    ax.set_title('AOI'+index[-1], fontsize=16)
    
    scalebar = AnchoredSizeBar(ax.transData, 1000, '1km', 'lower right',
                               pad=0.1, color='black', frameon=False, size_vertical=1,)
    ax.add_artist(scalebar)
    
#     aoi_buffer = buffer(index)
#     if isinstance(aoi_buffer, MultiPolygon):
#         ax.plot(*aoi_buffer.geoms[0].exterior.xy, c='r', label='Instrument footprint')
#         ax.plot(*aoi_buffer.geoms[2].exterior.xy, c='r')
#     elif isinstance(aoi_buffer, Polygon):
#         ax.plot(*aoi_buffer.exterior.xy, c='r', label='Instrument footprint')
    
    handles, labels = ax.get_legend_handles_labels()
    
legend_labels = {'#CD853F': 'Plateau', '#FFFF66': 'Valley', '#90EE90': 'Slopes', '#FFFFFF': '', }

patches = [Patch(color=color, label=label)
           for color, label in legend_labels.items()]
plt.legend(handles=patches+handles+[aoi_temp],
          bbox_to_anchor=(1.1, 1.05),
          facecolor='white',
          edgecolor='white',
          title='Topographic index:', fontsize=16)

plt.setp(ax.get_legend().get_title(), fontsize='16')

cbar_ax = fig.add_axes([0.2, 0.08, 0.6, 0.01])

clbr = fig.colorbar(obs_plot, cax=cbar_ax, orientation='horizontal')
clbr.ax.set_title('Brightness temperature 89GHz (K)', fontsize=16)
clbr.ax.tick_params(labelsize=14) 
    
fig.subplots_adjust(hspace=0.1, wspace=0.1)
plt.show()

fig.savefig('../Figs/Fig6.png', bbox_inches = 'tight')#, pad_inches = 0)

### Adding simulated TB to plots as well as obs:

In [ ]:
smrt_c087 = xr.open_dataset('../Other/smrt_results_atmos/smrt_atmos_tb_C087.nc')
smrt_c090 = xr.open_dataset('../Other/smrt_results_atmos/smrt_atmos_tb_C090.nc')

In [ ]:
vmin = min([np.nanmin(n) for n in [obs['M16-89'].TB.values, smrt_c087.sel(channel='M16-89')['base'].values]])
vmax = max([np.nanmax(n) for n in [obs['M16-89'].TB.values, smrt_c087.sel(channel='M16-89')['base'].values]])
norm_w_smrt = colors.Normalize(vmin=vmin, vmax=vmax)

In [ ]:
channel = 'M16-89'
flight = 'C087'
smrt = smrt_c087
cmap = ListedColormap(['#CD853F', '#FFFF66', '#90EE90', '#3399FF'])

for index, aoi in aoi_polygons.iterrows():
    fig, ax = plt.subplots(1,figsize=(8,8))

    pit_sub = pits[pits.index.str.contains(index)]
            
    for x, y, label in zip(pit_sub.geometry.x, pit_sub.geometry.y, pit_sub.index):
        ax.scatter(x,y,c=smrt.sel(channel=channel, snowpack=label)['base'],
                   s=200, cmap='plasma', norm=norm_w_smrt, edgecolor='lightgrey',
                   zorder=5, marker='h')

    obs_aoi = obs.loc[index][channel]
    obs_plot = ax.scatter(obs_aoi.geometry.x, obs_aoi.geometry.y, c=obs_aoi.TB, cmap='plasma', norm=norm_w_smrt)

    aoi_buffer = buffer(index)
    if isinstance(aoi_buffer, MultiPolygon):
        ax.plot(*aoi_buffer.geoms[0].exterior.xy, c='r', label='Instrument footprint')
        ax.plot(*aoi_buffer.geoms[2].exterior.xy, c='r')
    elif isinstance(aoi_buffer, Polygon):
        ax.plot(*aoi_buffer.exterior.xy, c='r', label='Instrument footprint')

    ax.plot(*aoi.geometry.exterior.xy, color='#0066CC')
    aoi_temp = Rectangle([0,0], 1,1, edgecolor='#0066CC', facecolor='none', linewidth=2, label='Areas of Interest')

    ymin, ymax = ax.get_ylim()
    xmin, xmax = ax.get_xlim()

    rasterio.plot.show(topography_rast, ax=ax, cmap=cmap, alpha=0.6)

    pit_sub = pits[pits.index.str.contains(index)]
    for x, y, label in zip(pit_sub.geometry.x, pit_sub.geometry.y, pit_sub.index):
        ax.annotate(label, xy=(x, y), xytext=(0, 10), textcoords="offset points", fontsize=12, zorder=10)
    
    asp_x_diff = (asp_x-(xmax-xmin))/2
    asp_y_diff = (asp_y-(ymax-ymin))/2
    
    ax.set_xlim(xmin-asp_x_diff, xmax+asp_x_diff)
    ax.set_ylim(ymin-asp_y_diff, ymax+asp_y_diff)

    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    scalebar = AnchoredSizeBar(ax.transData, 500, '500m', 'lower left',
                       pad=0.1, color='black', frameon=False, size_vertical=1,)
    ax.add_artist(scalebar)

    ax.set_title(index, fontsize=14)
    
    aoi_legend = Rectangle([0,0], 1,1, edgecolor='#0066CC', facecolor='none', linewidth=2, label='Areas of Interest')

    handles, labels = ax.get_legend_handles_labels()
    legend_labels = {'#CD853F': 'Plateau', '#FFFF66': 'Valley', '#90EE90': 'Slopes', '#FFFFFF': ''}

    patches = [Patch(color=color, label=label)
               for color, label in legend_labels.items()]

    plt.legend(handles=patches+handles+[aoi_legend],
          loc='upper left',
          facecolor='white',
          edgecolor='white',
          title='Topographic index:', fontsize=12)

    plt.setp(ax.get_legend().get_title(), fontsize='12')

    cbar_ax = fig.add_axes([0.5, 0.2, 0.38, 0.01])

    clbr = fig.colorbar(obs_plot, cax=cbar_ax, orientation='horizontal')
    clbr.ax.set_title('Brightness temperature 89GHz (K)', fontsize=12)
    clbr.ax.tick_params(labelsize=12) 
    
    plt.show()

In [ ]:
fig, axes = plt.subplots(3,3,figsize=(12,12))#, constrained_layout=True)

axes[2,2].remove()

channel = 'M16-89'
cmap = ListedColormap(['#CD853F', '#FFFF66', '#90EE90', '#3399FF'])

for (index, aoi), ax in zip(aoi_polygons.sort_index().iterrows(), axes.flatten()):  
    pit_sub = pits[pits.index.str.contains(index)]
    for x, y, label in zip(pit_sub.geometry.x, pit_sub.geometry.y, pit_sub.index):
        ax.scatter(x,y,c=smrt.sel(channel=channel, snowpack=label)['base'],
                   s=200, cmap='plasma', norm=norm_w_smrt, edgecolor='lightgrey',
                   zorder=5, marker='h')

    obs_aoi = obs[channel].loc[index]
    obs_plot = ax.scatter(obs_aoi.geometry.x, obs_aoi.geometry.y, c=obs_aoi.TB, cmap='plasma', norm=norm_w_smrt)
    
    ax.plot(*aoi.geometry.exterior.xy, color='#0066CC')
    aoi_temp = Rectangle([0,0], 1,1, edgecolor='#0066CC', facecolor='none', linewidth=2, label='Areas of Interest')

    ymin, ymax = ax.get_ylim()
    xmin, xmax = ax.get_xlim()
    
    rasterio.plot.show(topography_rast, ax=ax, cmap=cmap, alpha=0.8, label='Topographic index')

    asp_x_diff = (asp_x-(xmax-xmin))/2
    asp_y_diff = (asp_y-(ymax-ymin))/2
    
    ax.set_xlim(xmin-asp_x_diff, xmax+asp_x_diff)
    ax.set_ylim(ymin-asp_y_diff, ymax+asp_y_diff)
    
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    ax.set_title(index, fontsize=16)
    
    scalebar = AnchoredSizeBar(ax.transData, 1000, '1km', 'lower right',
                               pad=0.1, color='black', frameon=False, size_vertical=1,)
    ax.add_artist(scalebar)
    
    handles, labels = ax.get_legend_handles_labels()
    
legend_labels = {'#CD853F': 'Plateau', '#FFFF66': 'Valley', '#90EE90': 'Slopes', '#FFFFFF': '', }

patches = [Patch(color=color, label=label)
           for color, label in legend_labels.items()]
plt.legend(handles=patches+handles+[aoi_temp],
          bbox_to_anchor=(1.1, 1.05),
          facecolor='white',
          edgecolor='white',
          title='Topographic index:', fontsize=16)

plt.setp(ax.get_legend().get_title(), fontsize='16')

cbar_ax = fig.add_axes([0.2, 0.08, 0.6, 0.01])

clbr = fig.colorbar(obs_plot, cax=cbar_ax, orientation='horizontal')
clbr.ax.set_title('Brightness temperature 89GHz (K)', fontsize=16)
clbr.ax.tick_params(labelsize=14) 
    
fig.subplots_adjust(hspace=0.1, wspace=0.1)
# fig.tight_layout()
plt.show()